In [1]:
pip install numpy pandas scikit-learn



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\kapil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Load movies data
movies_df = pd.read_csv('C:/Users/kapil/Downloads/ml-25m/movies.csv', usecols=['movieId', 'title'], dtype={'movieId': 'int32', 'title': 'str'})
# Load ratings data
ratings_df = pd.read_csv('C:/Users/kapil/Downloads/ml-25m/ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})


C:\Users\kapil\AppData\Local\Temp\ipykernel_53280\1838691772.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
print(f"Original size: {ratings_df.shape}")
ratings_df = ratings_df.drop_duplicates(subset=['userId', 'movieId'])
print(f"Size after dropping duplicates: {ratings_df.shape}")


Original size: (25000095, 3)
Size after dropping duplicates: (25000095, 3)


In [4]:
print(ratings_df.dtypes)


userId       int32
movieId      int32
rating     float32
dtype: object


In [5]:
# Display the first few rows of the DataFrame
print(ratings_df.head())

# Check for any NaN values which might cause issues
print(ratings_df.isnull().sum())

# Summary statistics for numerical columns to identify any anomalies
print(ratings_df.describe())


   userId  movieId  rating
0       1      296     5.0
1       1      306     3.5
2       1      307     5.0
3       1      665     5.0
4       1      899     3.5
userId     0
movieId    0
rating     0
dtype: int64
             userId       movieId        rating
count  2.500010e+07  2.500010e+07  2.500010e+07
mean   8.118928e+04  2.138798e+04  3.533855e+00
std    4.679172e+04  3.919886e+04  1.060744e+00
min    1.000000e+00  1.000000e+00  5.000000e-01
25%    4.051000e+04  1.196000e+03  3.000000e+00
50%    8.091400e+04  2.947000e+03  3.500000e+00
75%    1.215570e+05  8.623000e+03  4.000000e+00
max    1.625410e+05  2.091710e+05  5.000000e+00


In [6]:
try:
    # Attempt a simple pivot without fill_value to see if it works
    simple_matrix = ratings_df.pivot_table(index='movieId', columns='userId', values='rating')
    print(simple_matrix.head())
except Exception as e:
    print(f"Error: {e}")


C:\Users\kapil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


Error: index 1007624404 is out of bounds for axis 0 with size 1007623835


In [7]:
# Check for duplicate (movieId, userId) pairs
duplicates = ratings_df.duplicated(subset=['movieId', 'userId']).sum()
print(f"Duplicate (movieId, userId) pairs: {duplicates}")


Duplicate (movieId, userId) pairs: 0


In [8]:
# Use a smaller subset of the DataFrame
small_df = ratings_df.sample(n=1000)  # Adjust n as needed based on your system's capabilities
try:
    small_matrix = small_df.pivot_table(index='movieId', columns='userId', values='rating', fill_value=0)
    print(small_matrix.head())
except Exception as e:
    print(f"Error with smaller subset: {e}")



userId   132     243     264     653     660     752     997     1156    \
movieId                                                                   
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
16          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

userId   1173    1456    ...  161277  161374  161470  161657  161658  161725  \
movieId                  ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
14          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0

In [9]:
# Example with made-up data
df_test = pd.DataFrame({
    'movieId': [1, 2, 3, 1, 2, 3],
    'userId': [1, 1, 1, 2, 2, 2],
    'rating': [4, 5, 3, 2, 3, 4]
})

try:
    matrix_test = df_test.pivot_table(index='movieId', columns='userId', values='rating', aggfunc='mean').fillna(0)
    print(matrix_test)
except Exception as e:
    print(f"Test Error: {e}")


userId     1    2
movieId          
1        4.0  2.0
2        5.0  3.0
3        3.0  4.0


In [10]:
import numpy as np
from scipy.sparse import csr_matrix

# Ensure userId and movieId are sequential integers starting from 0
# This step is necessary because scipy's sparse matrix constructors expect zero-indexed integers
user_ids = ratings_df['userId'].astype("category").cat.codes
movie_ids = ratings_df['movieId'].astype("category").cat.codes

# Number of users and movies
num_users = user_ids.nunique()
num_movies = movie_ids.nunique()

# Create a sparse matrix
ratings_matrix_sparse = csr_matrix((ratings_df['rating'], (movie_ids, user_ids)), shape=(num_movies, num_users))

# If you need to work with a dense matrix (e.g., for certain types of analyses), you can convert it back:
# Be cautious with this step if your dataset is large, as it can consume a lot of memory
ratings_matrix_dense = ratings_matrix_sparse.todense()

# Note: The dense matrix will have movies as rows and users as columns, with ratings as values


In [11]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between movies
# Note: cosine_similarity expects a dense matrix, but it can handle sparse matrices efficiently
movie_similarity = cosine_similarity(ratings_matrix_sparse, dense_output=False)

# movie_similarity is now a sparse matrix of shape (num_movies, num_movies) containing the cosine similarity between movies


In [12]:
def get_movie_title(movie_id, movies_df):
    """Helper function to get movie title by movieId"""
    return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]

def recommend_movies(movie_id, movies_df, movie_similarity, k=10):
    """
    Recommends movies based on a given movie ID.
    
    Parameters:
    - movie_id: The ID of the movie for which to find similar movies.
    - movies_df: DataFrame containing movie IDs and titles.
    - movie_similarity: Sparse matrix containing cosine similarity between movies.
    - k: Number of top similar movies to return.
    
    Returns:
    - A list of movie titles recommended for the user.
    """
    # Convert movie_id to the zero-based index used in the similarity matrix
    movie_idx = movies_df.index[movies_df['movieId'] == movie_id].tolist()[0]
    
    # Get similarity values with other movies
    # Since movie_similarity is a sparse matrix, toarray() will convert it to a dense format
    similarity_scores = movie_similarity[movie_idx].toarray().flatten()
    
    # Sort the movies based on similarity scores
    # [::-1] to sort in descending order, and [1:k+1] to skip the first movie (itself)
    similar_indices = similarity_scores.argsort()[::-1][1:k+1]
    
    # Convert back to movie IDs
    similar_movie_ids = movies_df.iloc[similar_indices]['movieId'].tolist()
    
    # Get movie titles
    similar_movies = [get_movie_title(id, movies_df) for id in similar_movie_ids]
    
    return similar_movies

# Example usage (assuming you have a movie_id and movies_df prepared)
movie_id = 1  # Toy Story; change this to a valid movieId from your dataset
recommended_movies = recommend_movies(movie_id, movies_df, movie_similarity, k=5)
print("Movies recommended:", recommended_movies)


Movies recommended: ['Star Wars: Episode IV - A New Hope (1977)', 'Toy Story 2 (1999)', 'Back to the Future (1985)', 'Forrest Gump (1994)', 'Jurassic Park (1993)']
